In [ ]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Step 2: Load the dataset
df = pd.read_csv('/content/superstores.csv')
df.columns = ['Month', 'Passengers']
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)

# Step 3: Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Step 4: Create sequences for LSTM
def create_sequences(data, time_step=10):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

time_step = 10
X, y = create_sequences(scaled_data, time_step)

# Step 5: Train-test split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Step 6: Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 7: Train the model
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1)

# Step 8: Make predictions
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

# Step 9: Inverse transform to get actual values
train_pred = scaler.inverse_transform(train_pred)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
test_pred = scaler.inverse_transform(test_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Step 10: Plot the results
plt.figure(figsize=(12,6))
plt.plot(df.index[time_step:split + time_step], y_train_inv, label='Train Actual')
plt.plot(df.index[time_step:split + time_step], train_pred, label='Train Predicted')
plt.plot(df.index[split + time_step:], y_test_inv, label='Test Actual')
plt.plot(df.index[split + time_step:], test_pred, label='Test Predicted')
plt.legend()
plt.title('Autism Forecast using LSTM')
plt.xlabel('Time')
plt.ylabel('Passengers')
plt.grid(True)
plt.show()

